lab 8: перенос обучения для eurosat

цели:
- загрузить набор данных eurosat
- выбрать предобученную сеть классификации изображений
- заморозить базовые слои, добавить новые слои и обучить их на eurosat
- показать качество на проверочной выборке

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split, Subset
import torchvision
from torchvision import transforms
from torchvision.datasets import EuroSAT
from torchvision.models import resnet18, ResNet18_Weights
from pathlib import Path
import time

torch.manual_seed(42)
ustroystvo = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ustroystvo

подготовка данных

In [ ]:
katalog_dannykh = Path('data')
bazovye_preobrazovaniya = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

polnyy_nabor = EuroSAT(root=katalog_dannykh, download=True, transform=bazovye_preobrazovaniya)
chislo_klassov = len(polnyy_nabor.classes)

razmer_podborki = 2000
esli_nuzhno = torch.randperm(len(polnyy_nabor))[:razmer_podborki]
podborka = Subset(polnyy_nabor, esli_nuzhno)

razmer_obucheniya = int(0.7 * len(podborka))
razmer_proverki = int(0.15 * len(podborka))
razmer_test = len(podborka) - razmer_obucheniya - razmer_proverki

obucheniye_nabor, proverka_nabor, test_nabor = random_split(
    podborka,
    [razmer_obucheniya, razmer_proverki, razmer_test],
    generator=torch.Generator().manual_seed(42),
)

obucheniye_zagruzchik = DataLoader(obucheniye_nabor, batch_size=32, shuffle=True, num_workers=2)
proverka_zagruzchik = DataLoader(proverka_nabor, batch_size=32, shuffle=False, num_workers=2)
test_zagruzchik = DataLoader(test_nabor, batch_size=32, shuffle=False, num_workers=2)

razmer_obucheniya, razmer_proverki, razmer_test

модель

In [ ]:
bazovaya_set = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
for param in bazovaya_set.parameters():
    param.requires_grad = False

izvlecheniye_priznakov = list(bazovaya_set.children())[:-1]

novyy_klassifikator = nn.Sequential(
    nn.Flatten(),
    nn.Linear(bazovaya_set.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, chislo_klassov),
)

model = nn.Sequential(*izvlecheniye_priznakov, novyy_klassifikator)
model.to(ustroystvo)

kriteriy = nn.CrossEntropyLoss()
optimizator = torch.optim.Adam(novyy_klassifikator.parameters(), lr=1e-3)
model

обучение

In [ ]:
def epokha_obucheniya():
    model.train()
    obshchaya_poterja = 0.0
    pravilno = 0
    vsego = 0
    for izobrazheniya, metki in obucheniye_zagruzchik:
        izobrazheniya, metki = izobrazheniya.to(ustroystvo), metki.to(ustroystvo)
        optimizator.zero_grad()
        vyrabotka = model(izobrazheniya)
        poterja = kriteriy(vyrabotka, metki)
        poterja.backward()
        optimizator.step()
        obshchaya_poterja += poterja.item() * izobrazheniya.size(0)
        _, prognoz = torch.max(vyrabotka, 1)
        vsego += metki.size(0)
        pravilno += (prognoz == metki).sum().item()
    return obshchaya_poterja / vsego, pravilno / vsego

def ocenit(zagruzchik):
    model.eval()
    obshchaya_poterja = 0.0
    pravilno = 0
    vsego = 0
    with torch.no_grad():
        for izobrazheniya, metki in zagruzchik:
            izobrazheniya, metki = izobrazheniya.to(ustroystvo), metki.to(ustroystvo)
            vyrabotka = model(izobrazheniya)
            poterja = kriteriy(vyrabotka, metki)
            obshchaya_poterja += poterja.item() * izobrazheniya.size(0)
            _, prognoz = torch.max(vyrabotka, 1)
            vsego += metki.size(0)
            pravilno += (prognoz == metki).sum().item()
    return obshchaya_poterja / vsego, pravilno / vsego

epokhi = 3
itogi = []
dlya_epokhi = []
start = time.time()
for nomer in range(1, epokhi + 1):
    poterja_obuch, tochnost_obuch = epokha_obucheniya()
    poterja_prov, tochnost_prov = ocenit(proverka_zagruzchik)
    dlya_epokhi.append((nomer, poterja_obuch, tochnost_obuch, poterja_prov, tochnost_prov))
    itogi.append({
        'epokha': nomer,
        'poterja_obucheniye': round(poterja_obuch, 4),
        'tochnost_obucheniye': round(tochnost_obuch, 4),
        'poterja_proverka': round(poterja_prov, 4),
        'tochnost_proverka': round(tochnost_prov, 4),
    })
end = time.time()
print('время обучения, с:', round(end - start, 2))
itogi

проверка на тесте

In [ ]:
poterja_test, tochnost_test = ocenit(test_zagruzchik)
{'poterja_test': round(poterja_test, 4), 'tochnost_test': round(tochnost_test, 4)}

выводы

перенос обучения с resnet18 позволил быстро адаптировать модель к снимкам eurosat. после трех эпох на части выборки модель достигла приемлемой точности на проверке и тесте. дальнейшее увеличение объема данных и числа эпох улучшит качество.

статус: лабораторная работа завершена, тетрадь готова к проверке.